In [1]:
import socket
import time
import struct
import numpy as np
import pynq
from pynq import MMIO

# Define a fixed-point data type with 16 bits total and 6 bits for the integer part


def float_to_fixed_point(floating_number, num_integer_bits=6, num_fractional_bits=18):
    # Clamp the floating number to the maximum and minimum representable range
    max_val = 2**(num_integer_bits - 1) - 2**-num_fractional_bits
    min_val = -2**(num_integer_bits - 1)
    clamped_float = max(min(floating_number, max_val), min_val)

    # Scale the floating-point number by 2^num_fractional_bits and round it
    scaled_number = round(clamped_float * (2**num_fractional_bits))

    # Convert the scaled number to a 24-bit two's complement representation
    fixed_point_number = scaled_number & ((1 << (num_integer_bits + num_fractional_bits)) - 1)

    return fixed_point_number

def fixed_point_to_float(fixed_point_number, num_integer_bits=6, num_fractional_bits=18):
    # Check if the number is negative by looking at the sign bit
    sign_bit_mask = 1 << (num_integer_bits + num_fractional_bits - 1)
    is_negative = fixed_point_number & sign_bit_mask

    # Convert the fixed-point number to a signed integer if it is negative
    if is_negative:
        fixed_point_number = fixed_point_number - (1 << (num_integer_bits + num_fractional_bits))

    # Convert the fixed-point number back to a floating-point number by scaling down
    floating_number = fixed_point_number / (2**num_fractional_bits)

    return floating_number



overlay = pynq.Overlay('actor_network.bit')
top_ip = overlay.actor_network_0
top_ip.signature
# The float_list now contains the original floats

# Create a client socket
client_socket = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
server_address = ('192.168.2.1', 9090) # Replace with the IP address of the host PC
client_socket.connect(server_address)

x = []

a_buffer = pynq.allocate((11,), dtype = np.int32)
sum_buffer = pynq.allocate((3), dtype = np.int32)

    
for i in range(10000):
    received_x_bytes = client_socket.recv(64)
    x = []
    for i in range(0, len(received_x_bytes), 4):
        x.append(struct.unpack('f', received_x_bytes[i:i+4])[0])
    print(x)
    
    for i in range(0, len(x)):
        a_buffer[i] = float_to_fixed_point(x[i])
    aptr = a_buffer.physical_address
    sumptr = sum_buffer.physical_address
    
    top_ip.write(0x10, aptr)
    top_ip.write(0x1c, sumptr)
    top_ip.write(0x00, 1)
    isready = top_ip.read(0x00)
    
    while (isready == 1):
        isready = top_ip.read(0x00)
    print(sum_buffer)
    
    x_bytes = b''
    for f in sum_buffer:
        x_bytes += struct.pack('f', fixed_point_to_float(f))

    client_socket.send(x_bytes)

#print(x)
client_socket.close()


[1.248080849647522, -0.0011519562685862184, 0.0024627933744341135, -0.004569766111671925, 0.0014587389305233955, 0.004747253376990557, 0.0039872596971690655, 0.0026315608993172646, 0.0009625294478610158, 0.0026605776511132717, 0.0029596781823784113]
[248016 254842 218285]
[1.2476476430892944, 0.0013475125888362527, 0.0025097718462347984, 0.0003876244300045073, 0.005972427316009998, 0.13529954850673676, -0.1118130013346672, 0.5803884863853455, 0.009895813651382923, 1.148715615272522, 1.1289981603622437]
[  248355 16574088   244819]
[1.246371865272522, 0.004132224712520838, 0.0026345248334109783, 0.005127157084643841, 0.020571088418364525, 0.06256185472011566, -0.20670358836650848, 0.11471254378557205, 0.019591040909290314, 0.03773246333003044, 2.5189387798309326]
[  241041 16525071   218515]
[1.2443642616271973, 0.0027032827492803335, 0.002800974063575268, 4.7029712732182816e-05, 0.04572591558098793, -0.03532833978533745, -0.29563429951667786, -0.4720859229564667, 0.02124611660838127, -

BrokenPipeError: [Errno 32] Broken pipe